In [1]:
# pip install "gymnasium[box2d]" pyglet

In [2]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os
import torch 

device = torch.device("mps" if torch.mps.is_available() else "cpu")

In [3]:
enviorment_name = "CarRacing-v3"
env = gym.make(enviorment_name, render_mode="human")

In [4]:
env.reset()

2024-12-20 16:32:18.753 Python[12089:386545] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-20 16:32:18.753 Python[12089:386545] +[IMKInputSession subclass]: chose IMKInputSession_Modern


(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 {})

In [5]:
env.action_space

Box([-1.  0.  0.], 1.0, (3,), float32)

In [6]:
env.observation_space

Box(0, 255, (96, 96, 3), uint8)

In [7]:
env.reset()
env.render() 

In [8]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward

    print(f'Episode:{episode}, Score:{score}')
env.close() 

Episode:1, Score:-34.210526315789984
Episode:2, Score:-33.333333333333805
Episode:3, Score:-29.328621908127644
Episode:4, Score:-41.17647058823595
Episode:5, Score:-31.97278911564679


In [9]:
env = gym.make(enviorment_name)
env = DummyVecEnv([lambda: env])

In [10]:
log_path = os.path.join('Training', 'Logs')
model = PPO("CnnPolicy", env, verbose=1, tensorboard_log=log_path, device=device)


Using mps device
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(total_timesteps=400000)

Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 130  |
|    iterations      | 1    |
|    time_elapsed    | 15   |
|    total_timesteps | 2048 |
-----------------------------


In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model_CarRacing')
model.save(PPO_path)